# Semester Project Notebook - EPFL HP

In [2]:
# Libraries
import pandas as pd
import numpy as np

# 01 - Fluid selection

### Legislation: banned fluids (Montreal Protocol, OFEV)
Fluid with ozone depletion potention, global warming potential and stable in the atmosphere.
- HCFC 
- HCFO
- CFC 
- HFC 

### Environmental Criteria
- GWPmax = 400 ?
- ODPmax = 0

### Operating Criteria
- Tcrit,min  = 90°C (to avoid entering the supercritical domain)
- Teb,max = 10°C (if Teb too high, very low pressure required for evaporation)


### Fluid screening
Selected for the study: Fluids figuring in the (non-exhaustive) OFEV's list of autorized fluids:
- R-290 (propane, HC)
- R-600 (butane, HC)
- R-600a (isobutane, HC)
- R-1270 (propylène, HO)
- R-1234yf (HFO)
- R-1234ze (HFO)
- R-717 (ammonia, N)

Not selected for the study: Fluids not in the list:
- R-630 (methylamine, HC)
- R-40 (HCC)
- R-1140 (HCO)
- R-764 (sulfure dioxide, N)

### Physical properties for unknown fluids in Belsim VALI (R-1234yf, R-1234ze)
- Thermodynamic models: VIDEAL/LIDEAL or Soave/Peng-Robinson
- Physical properties required: critical temperature, critical pressure, boiling curve coefficients, ascentric factor

In [16]:
properties = {'ASHRAE': ['R-1234yf','R-1234ze'], 'Formula':['C3F4H2','C3F4H2'], 'Tcrit [K]':[367.85,423.27], 'Pcrit [bar]':[33.822, 35.349], 'PVAP':['?','?'], 'omega':[0.276,0.326]}
properties = pd.DataFrame(properties)
properties = properties.set_index('ASHRAE')
properties

,Formula,Tcrit [K],Pcrit [bar],PVAP,omega
ASHRAE,,,,,
R-1234yf,C3F4H2,367.85,33.822,?,0.276
R-1234ze,C3F4H2,423.27,35.349,?,0.326


# 02 - Heating demand

Monthly mean heating demand during operating hours for the two heating loops (lowT (50°C) and mediumT (65°C)) of EPFL's heating system. 

In [35]:
heating_demand = pd.read_csv('Data/heating_monthly.csv', sep=';')
heating_demand = heating_demand.set_index('Month')
heating_demand['MT'] = heating_demand['MT'].apply(lambda x: x/1000 if x > 0 else 0)
heating_demand['LT'] = heating_demand['LT'].apply(lambda x: x/1000 if x > 0 else 0)
heating_demand.rename(columns={'MT':'MT [kW]', 'LT':'LT [kW]'})

,MT [kW],LT [kW]
Month,,
1,7395.670705,4318.682087
2,5534.986003,3283.383266
3,2904.392606,1811.057933
4,43.868802,192.982501
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000
8,0.000000,0.000000
9,0.000000,36.520003


# 03 - Optimal condensation temperature (HP_T_6)

### Working assumptions
- HP design optimized for colder working conditions (Q_ cond = 6 MW)
- isentropic compressor efficiency: 0.85
- same pressure ratio for the 2 compressors
- boiling temperature at the evaporator is constant (6°C)
- water enters in EPFL with T_EPFL_in = 65/50 °C and leaves EPFL with T_EPFL_out = 30°C
- CO2 emissions from gird electricity: 0.206 kgCO2eq/kWh
- swiss CO2 tax: 100 CHF/t CO2
- electricity price: 0.0916 CHF/kWh
- lifetime = 20 years

### Flowchart of the Belsim VALI Model
Imposed conditions (CST) are marked in yellow

<img src=flowchart_optimal_t_hp_6.png>

### Compressor and heat exchanger cost calculation (CAPEX)

$A_{HEX} = \frac{Q_{cond}}{U \cdot LMDT} $

$C_p = 10^{K_1 + K_2\cdot \log(A) + K_3 \cdot \log(A)^2}$
where $A$ is the HEX area in [m2] or the compressor power in [kW].

$C = F_{BM} \cdot C_p$

$CAPEX = C \cdot \frac{i(i+1)^n}{(i+1)^n-1}$ [CHF/year]

In [40]:
cost_parameters = {'Equipment': ['Compressor','Condensor','Evaporator'], 'Fbm':[2.45,4.74,4.74], 'K1':[2.2897,3.2138,3.5638], 'K2':[1.3604,0.2688,0.1906], 'K3':[-0.1027,0.7961,0.1107], 'Iref':[584.7,394.3,394.3]}
cost_parameters = pd.DataFrame(cost_parameters)
cost_parameters = cost_parameters.set_index('Equipment')
cost_parameters

,Fbm,K1,K2,K3,Iref
Equipment,,,,,
Compressor,2.45,2.2897,1.3604,-0.1027,584.7
Condensor,4.74,3.2138,0.2688,0.7961,394.3
Evaporator,4.74,3.5638,0.1906,0.1107,394.3


### Operation costs (OPEX)

$OPEX = \sum_i^{period} E_i (c_{el} + grid_{CO_2} \cdot tax_{CO2})$

### Results from VALI
Iteration of the model over T_HP_6 for each typical period

In [6]:
result_VALI = pd.read_csv('Data/R-717_all_Results.csv', sep=',')
result_VALI

,month,W_COMP_1,W_COMP_2,Q_COND_LOAD,Q_EVAP_LOAD,T_HP_5,T_HP_6
0,1,391.017,465.296,6000.0,5143.69,67.1213,40
1,1,400.651,479.268,6000.0,5120.08,68.8302,41
2,1,410.185,493.260,6000.0,5096.56,70.5342,42
3,1,419.620,507.273,6000.0,5073.11,72.2333,43
4,1,428.957,521.307,6000.0,5049.74,73.9276,44
...,...,...,...,...,...,...,...
177,12,573.136,763.783,6000.0,4663.08,102.0100,61
178,12,580.786,778.317,6000.0,4640.90,103.6210,62
179,12,588.346,792.885,6000.0,4618.77,105.2270,63
180,12,595.817,807.490,6000.0,4596.69,106.8290,64


### Computations and plot for optimization

In [26]:
#Condenser design for maximal load
T_EPFL_MT_in = 65
T_EPFL_LT_in = 50
T_EPFL_out = 30
data_max_load = result_VALI[result_VALI['month']==1]
data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)

data_max_load

<ipython-input-26-adf5891c59bf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)


,month,W_COMP_1,W_COMP_2,Q_COND_LOAD,Q_EVAP_LOAD,T_HP_5,T_HP_6,LMTD
0,1,391.017,465.296,6000.0,5143.69,67.1213,40,5.081210
1,1,400.651,479.268,6000.0,5120.08,68.8302,41,6.796159
2,1,410.185,493.260,6000.0,5096.56,70.5342,42,8.354183
3,1,419.620,507.273,6000.0,5073.11,72.2333,43,9.836521
4,1,428.957,521.307,6000.0,5049.74,73.9276,44,11.274262
5,1,438.197,535.364,6000.0,5026.44,75.6170,45,12.682522
6,1,447.339,549.443,6000.0,5003.22,77.3017,46,14.069935
7,1,456.385,563.545,6000.0,4980.07,78.9815,47,15.441610
8,1,465.334,577.671,6000.0,4956.99,80.6566,48,16.801071
9,1,474.188,591.821,6000.0,4933.99,82.3269,49,18.150600
